In [3]:
import numpy as np
import pandas as pd
import itertools as it
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score,confusion_matrix,recall_score,accuracy_score
from sklearn.metrics import roc_curve, auc,precision_recall_curve, roc_curve
import matplotlib.pylab as plt

## Carga de Datos

In [4]:
DataIn = pd.read_excel('Data.xlsx')
DataIn = DataIn.set_index('Date')
Data = DataIn.dropna(axis=0,how='any')
Y = Data['Class']
X = Data.drop('Class', axis = 1)

In [5]:
Data = pd.read_excel('Data.xlsx')
Data = Data.set_index('Date')

# Definición de funciones para optimización de hyper parametros

## Parametrización 

In [14]:
Epsilon = 0.2
delta = 0.1
Hyper_Parameters = [['l1','l2'],[100,150,200]] 
set_parameters = [i for i in it.product(*Hyper_Parameters)]
sample_size = X.shape[0]
training_sample_size = int(sample_size * 0.6)
validation_sample_size = int(sample_size * 0.2)
test_sample_size = validation_sample_size

## División de datos en: Entrenamiento, Pruebas, Validación.

*Dividimos la muestra en 60 % de Entrenamiento, 20% de Pruebas y 20% de Validación.*


- n: Número mínimo optimo/estimado de elementos para que el modelo aprenda. Serán los datos que utilizaremos para entrenar el modelo. Se sacan del 60% total de la muestra.
- validation_data: 20 % del total de la muestra.

In [18]:
def split_data_for_model_training(X, Y, training_sample_size, validation_sample_size):
    X_tr, X_ts, Y_tr, Y_ts = train_test_split(X,
                                              Y, 
                                              train_size = training_sample_size, 
                                              test_size = validation_sample_size*2, 
                                              random_state=19)
    
    X2_va, X2_ts, Y2_va, Y2_ts = train_test_split(X_ts, 
                                                  Y_ts, 
                                                  train_size = 0.5, 
                                                  random_state=19)
    return X_tr, Y_tr, X2_va, Y2_va, X2_ts, Y2_ts

In [22]:
def building_a_logistic_model(validation_data_x, validation_data_y,
                          training_data_x, training_data_y,
                          n, penalty, max_iter):
    
    log = LogisticRegression(penalty=penalty, max_iter=max_iter)
    X_tr, X_ts, Y_tr, Y_ts = train_test_split(training_data_x, 
                                              training_data_y, 
                                              train_size = n,
                                              random_state=19)
    
    log.fit(X_tr, Y_tr)
    y_pred = log.predict(validation_data_x)
    a_score = accuracy_score(validation_data_y, y_pred)
    p_score = precision_score(validation_data_y, y_pred, average = 'macro')
    r_score = recall_score(validation_data_y, y_pred, average = 'macro')
    return log, y_pred, a_score, p_score, r_score

## Cálculo de n min optimo:


In [23]:
def optim_n_for_tree(features):
    return features+1

# Modelos - N mínimo Optimo

Aqui encontramos la regresion logistica con el score mas alto y mas bajo para todos los n optimos de las clases de hipotesis

In [24]:
def main_fun(epsilon, delta, validation_data_x, validation_data_y,
                          training_data_x, training_data_y, _set_parameters, n):
    results = []
    # TODO - Add Try/Catch
    for p in _set_parameters:              
        # Validamos si n esta dentro del sample
        if n > len(training_data_x):
            return results
        else:             
            log, y_pred, a_score, p_score, r_score =  building_a_logistic_model(validation_data_x, validation_data_y,
                                    training_data_x, training_data_y,
                                    n,
                                    p[0], 
                                    p[1])
            
            results.append((log, y_pred, a_score, p_score, r_score, n))
    return results

In [25]:
X_tr, Y_tr, X2_va, Y2_va, X2_ts, Y2_ts  = split_data_for_model_training(X, Y, training_sample_size, validation_sample_size)

D:\Usuarios\juanespe\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [30]:
n = optim_n_for_tree(X.shape[1])
w = main_fun(Epsilon, delta,X2_ts, Y2_ts, X_tr, Y_tr, set_parameters, n)

D:\Usuarios\juanespe\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
D:\Usuarios\juanespe\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Usuarios\juanespe\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
D:\Usuarios\juanespe\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, w

In [31]:
max(w, key = lambda t: t[2])

(LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='warn',
           n_jobs=None, penalty='l1', random_state=None, solver='warn',
           tol=0.0001, verbose=0, warm_start=False),
 array([ 0,  0,  0, -1,  0, -1,  0,  0, -1,  0, -1,  0,  0, -1,  0,  0,  0,
        -1,  0, -1, -1, -1, -1,  0, -1, -1,  0,  0, -1,  0, -1, -1,  0, -1,
         0,  0,  0,  0, -1, -1,  0, -1,  0, -1, -1,  0,  0, -1,  0, -1,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1, -1,  0, -1, -1,  0, -1,
         0,  0,  0,  0, -1,  0, -1,  0, -1,  0,  0,  0, -1,  0, -1,  0, -1,
         0,  0,  0, -1, -1,  0,  0, -1,  0, -1,  0,  0, -1,  0,  0,  0,  0,
        -1,  0, -1,  0,  0,  0,  0,  0,  0, -1,  0,  0, -1,  0,  0, -1, -1,
        -1,  0,  0,  0,  0, -1, -1,  0,  0,  0, -1, -1,  0,  0, -1,  0, -1,
        -1,  0,  0, -1, -1, -1,  0,  0, -1,  0, -1, -1, -1,  0,  0,  0, -1,
         0,  0, -1,  0,  0,  0,  0, -1,  0, -1,

In [34]:
min(w, key = lambda t: t[2])

(LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='warn',
           n_jobs=None, penalty='l2', random_state=None, solver='warn',
           tol=0.0001, verbose=0, warm_start=False),
 array([ 0,  0,  0, -1,  0, -1,  0,  0,  0,  0, -1, -1,  0, -1,  0, -1,  0,
        -1,  0, -1, -1,  0, -1,  0, -1,  0, -1,  0, -1,  0, -1, -1,  0, -1,
         0,  0,  0,  0, -1,  0,  0, -1,  0, -1, -1,  0,  0, -1,  0, -1,  0,
         0,  0,  0,  0,  0,  1,  0,  0, -1,  0, -1, -1, -1, -1, -1,  0, -1,
         0,  0,  0,  0, -1,  0, -1, -1, -1, -1,  0,  0,  0,  0, -1,  0, -1,
         0, -1,  1,  0, -1,  0,  0, -1,  0,  0,  0,  1,  0,  0,  0,  0,  0,
        -1,  0, -1,  0,  0,  0,  0,  0, -1, -1,  0,  0, -1,  0,  0, -1, -1,
        -1, -1,  0, -1,  0, -1, -1,  0,  0,  0, -1,  0,  0,  0, -1,  0, -1,
        -1,  0,  0, -1, -1, -1,  0,  0, -1,  0, -1, -1, -1,  0,  0,  0, -1,
         0,  0, -1, -1, -1,  0,  0,  0,  0,  0,